In [1]:
from tkinter import *
import cv2 
from PIL import Image, ImageTk 
import numpy as np
import time
import math
import sys
from pyfirmata import Arduino

In [8]:
# Seting up pyfirmata params
board = Arduino('/dev/tty.HC-05')

# Left motor pins
enableLeftPin = board.get_pin('d:11:p')
input1LeftPin = board.get_pin('d:13:o')
input2LeftPin = board.get_pin('d:12:o')

# Right motor pins
enableRightPin = board.get_pin('d:10:p')
input3RightPin = board.get_pin('d:9:o')
input4RightPin = board.get_pin('d:8:o')


NameError: name 'board' is not defined

In [ ]:
# Setting Aruco params

# Pruned aruco type dict. See PoseEstimation.ipynb for full list
ARUCO_DICT = {"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL}

arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT["DICT_ARUCO_ORIGINAL"])

arucoParams = cv2.aruco.DetectorParameters_create()


intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948,0.18514,0,0))

In [ ]:
# defining  functions

def setMotorSpeeds(leftSpeed, rightSpeed):
        # Set left motor direction
        if leftSpeed >= 0:
            input1LeftPin.write(1)
            input2LeftPin.write(0)
        else:
            input1LeftPin.write(0)
            input2LeftPin.write(1)
            leftSpeed = -leftSpeed
      
        # Set right motor deirection
        if rightSpeed >= 0:
            input3RightPin.write(1)
            input4RightPin.write(0)
        else:
            input3RightPin.write(0)
            input4RightPin.write(1)
            rightSpeed = -rightSpeed
    
        enableLeftPin.write(leftSpeed/255)
        enableRightPin.write(rightSpeed/255)

def calcMotorSpeed_PID_TupleIn(currentPoseTuple, desiredPositionTuple, CCW_ANGLE_DIRECTION = 1, integralError = 0, derivativeError = 0):
    '''
    Parameters
    ----------
    robotPoseTuple : Tuple of floats
        (x, y, phi) Tuple describing the robot's current position and heading: .
    desiredPointTuple : Tuple of floats
        (x_desired, y_desired) Position of robot's current point it's tracking towards.
    CCW_ANGLE_DIRECTION: 1 or -1, default is 1
        Input -1 if rotating CCW decreases angle reading
    integralError : float, optional
        Integral Error, whatever way you like to calculate. The default is 0.
    derivativeError : float, optional
        Derivative Error, whatever way you like to calculate. The default is 0.

    Returns
    -------
    motorSpeedTuple : Tuple of Floats
        (leftSpeed, rightSpeed) speeds as float vary from -255 to 255.
    '''
    # Process Input
    x = currentPoseTuple[0];
    y = currentPoseTuple[1];
    phi = currentPoseTuple[2];
    x_desired = desiredPositionTuple[0];
    y_desired = desiredPositionTuple[1];
    # Constant Parameters: 
    kp = 8 #proportional constant
    ki = 0 #integralError constant
    kd = 0 #derivativeError constant
    ANGLE_OF_GOING_RIGHT = -90; #angle (deg) the robot detects when moving right. Set to -90 if up is 0, CCW is positive.
    DEFAULT_SPEED = 255 #the regular speed the robot runs at
    MAX_SPEED = 255 #exactly 255
    MIN_SPEED = -255;
    CCW_ANGLE_DIRECTION = 1; #set to 1 if CCW is negative
    ######

    # Same code as before
    dx = x_desired - x;
    dy = y_desired - y;
    leftSpeed = DEFAULT_SPEED
    rightSpeed = DEFAULT_SPEED
    angleToDesired = math.degrees(math.atan2(dy, dx)); #
    currentAngle = phi - ANGLE_OF_GOING_RIGHT
    
    #Positive error means need to move CCW
    error = angleToDesired - currentAngle;
    error = CCW_ANGLE_DIRECTION * error;
    
    u = kp * error + ki * integralError + kd * derivativeError
    
    leftSpeed = max(MIN_SPEED, min(MAX_SPEED, leftSpeed - u)) 
    rightSpeed = max(MIN_SPEED, min(MAX_SPEED, rightSpeed + u))
    motorSpeedTuple = (leftSpeed, rightSpeed)
    return motorSpeedTuple #return a tuple, of the new motor speeds

def generate_lawnmower_path(xTopLeft, yTopLeft, xBottomRight, yBottomRight, L, ds=2):
    '''
    Parameters
    ----------
    xTopLeft : float
        X coordinate of Top Left (or bottom left) point of bounding box.
    yTopLeft : float
        Y coordinate of Top Left (or bottom left) point of bounding box.
    xBottomRight : float
        X coordinate of Bottom Right (or top right) point of bounding box.
    yBottomRight : float
        X coordinate of Bottom Right (or top right) point of bounding box.
    L : float
        Distance between zig zags. Input the width of the robot or something.
    ds : float, optional
        The distance between points. The default is 2 units.

    Returns
    -------
    points : List of tuples, where each row/tuple is (x,y) of that point along path
        DESCRIPTION.

    '''
    points = []
    x_current = xTopLeft
    y_current = yTopLeft

    width = xBottomRight - xTopLeft
    height = yBottomRight - yTopLeft

    # Ensure that height is positive, assuming the direction is downward
    if height < 0:
        height = -height
        y_direction = -1
    else:
        y_direction = 1

    direction = 1  # 1 for right, -1 for left

    while (y_direction == 1 and y_current < yBottomRight) or (y_direction == -1 and y_current > yBottomRight):
        # Horizontal movement
        if direction == 1:
            while x_current < xBottomRight:
                points.append((x_current, y_current))
                x_current += ds
                if x_current >= xBottomRight:
                    x_current = xBottomRight
                    points.append((x_current, y_current))
                    break
        else:
            while x_current > xTopLeft:
                points.append((x_current, y_current))
                x_current -= ds
                if x_current <= xTopLeft:
                    x_current = xTopLeft
                    points.append((x_current, y_current))
                    break

        # Vertical movement
        y_next = y_current + y_direction * L
        if y_direction == 1 and y_next >= yBottomRight:
            y_next = yBottomRight
        elif y_direction == -1 and y_next <= yBottomRight:
            y_next = yBottomRight
        while (y_direction == 1 and y_current < y_next) or (y_direction == -1 and y_current > y_next):
            y_current += y_direction * ds
            if (y_direction == 1 and y_current > y_next) or (y_direction == -1 and y_current < y_next):
                y_current = y_next
            points.append((x_current, y_current))

        direction *= -1

    return points

# output coords of both 0 and 153
# adjust loop to accommodate
def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.Dictionary_get(aruco_dict_type)
    parameters = cv2.aruco.DetectorParameters_create()

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters,
                                                                cameraMatrix=matrix_coefficients,
                                                                distCoeff=distortion_coefficients)

    if len(corners) > 0:
        for i in range(0, len(ids)):
            if ids[i] in [0, 153]:
                rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients,
                                                                               distortion_coefficients)
                cv2.aruco.drawDetectedMarkers(frame, corners)
                cv2.aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)
                print(f"ArUco marker ID: {ids[i]}, Coordinates: {tvec}")

    return frame

# Not used (yet?) see ArucoDetect.ipynb for demo
def aruco_display(corners, ids, rejected, image):
    if len(corners) > 0:
        ids = ids.flatten()
        for (markerCorner, markerID) in zip(corners, ids):
            corners = markerCorner.reshape((4, 2))
            (topLeft, topRight, bottomRight, bottomLeft) = corners

            topRight = (int(topRight[0]), int(topRight[1]))
            bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
            bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
            topLeft = (int(topLeft[0]), int(topLeft[1]))

            cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
            cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
            cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
            cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)

            cX = int((topLeft[0] + bottomRight[0]) / 2.0)
            cY = int((topLeft[1] + bottomRight[1]) / 2.0)
            cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)

            cv2.putText(image, str(markerID), (topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)
            print("[Inference] ArUco marker ID: {}".format(markerID))

    return image

# Function for capturing video frame and displaying it.
# Schedules itself to run again in 10ms
# This will probably be our loop
def open_camera():
    global initiate_cleaning, cleaning, pts, i, x1, y1, x2, y2
    # Capture the video frame by frame 
    _, frame = vid.read()
    
    # Paul's Pose Estimation goes here.
    
    
    # Draw the bounding box
    try:
        x1_int = int(x1_var.get())
        y1_int = int(y1_var.get())
        x2_int = int(x2_var.get())
        y2_int = int(y2_var.get())
        cv2.rectangle(frame, (x1_int, y1_int), (x2_int, y2_int), (255, 0, 0), 2)
    except ValueError:
        pass  # Handle the case where the inputs are not valid integers
    
    # Convert image from one color space to other 
    opencv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    
    # Capture the latest frame and transform to image 
    captured_image = Image.fromarray(opencv_image)

    # Convert captured image to photoimage 
    photo_image = ImageTk.PhotoImage(image=captured_image)

    # Displaying photoimage in the label 
    label_widget.photo_image = photo_image

    # Configure image in the label 
    label_widget.configure(image=photo_image)


    robotWidth = 20
    # Run the following if the "Clean" button has been clicked
    if initiate_cleaning and not cleaning:
        pts = generate_lawnmower_path(x1_int, y1_int, x2_int, y2_int, 0.75*robotWidth, ds = 2)
        i = 0
        initiate_cleaning = False
        cleaning = True

    if cleaning:
        # if dist to pts(i) < threshold
            # if i == length(pts): cleaning = False, motor speeds = 0
            # else: i++, reset error vals
        
        robotPos = (0, 0, 0)
        pt = (1, 1)
        
        # run PID
        #(leftSpeed, rightSpeed) = calcMotorSpeed_PID(robotPos, pt, 0, 0)
        (leftSpeed, rightSpeed) = calcMotorSpeed_PID_TupleIn(robotPos, pt)
        # print(leftSpeed)
        # print(rightSpeed)
        setMotorSpeeds(leftSpeed, rightSpeed)

    # Repeat the same process after every 10 seconds 
    label_widget.after(10, open_camera)


In [ ]:
# Define a video capture object 
vid = cv2.VideoCapture(0) 

# Declare the width and height in variables 
width, height = 800, 600

# Set the width and height 
# adjust to match phone resolution
vid.set(cv2.CAP_PROP_FRAME_WIDTH, width) 
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, height) 

# Create a GUI app 
app = Tk() 

# Bind the app with Escape keyboard to 
# quit app whenever pressed 
app.bind('<Escape>', lambda e: app.quit()) 

# Create a label and display it on app 
label_widget = Label(app) 
label_widget.pack() 

# Create a button to open the camera in GUI app
button1 = Button(app, text="Open Camera", command=open_camera)
button1.pack()

# Create input fields for user-defined cleaning boundaries
x1_var = StringVar()
y1_var = StringVar()
x2_var = StringVar()
y2_var = StringVar()

entry_x1 = Entry(app, textvariable=x1_var)
entry_y1 = Entry(app, textvariable=y1_var)
entry_x2 = Entry(app, textvariable=x2_var)
entry_y2 = Entry(app, textvariable=y2_var)

entry_x1.pack()
entry_y1.pack()
entry_x2.pack()
entry_y2.pack()

# Function to set cleaning initiation
def start_cleaning():
    global initiate_cleaning
    initiate_cleaning = True

# Create button to control set bool initiate_cleaning = True
start_button = Button(app, text="Start Cleaning", command=start_cleaning)
start_button.pack()

# Initialize state as not cleaning
cleaning = False  # Usually False
initiate_cleaning = False

# Initialize array of desired pts and all control variables
pts = []
i = 0

# Create an infinite loop for displaying app on screen
app.mainloop()